## Используем большие языковые модели в VK Cloud

Для работы над заданием мы предлагаем вам использовать три больших языковых модели, развёрнутых на узлах с GPU в VK Cloud:

* **Внутренняя экспериментальная модель Mail.ru**: `http://hackllm.vkcloud.eazify.net:8000`
* Модель **Saiga3 8b**: `http://saiga.vkcloud.eazify.net:8000`
* Модель **Saiga Gemma 9b**: `http://shwars.vkcloud.eazify.net:8000` (не поддерживает интерфейс llm)

Для начала установим все необходимые библиотеки. Мы будем использовать LangChain для вызова языковых моделей по OpenAI-совеместимому интерфейсу.

In [6]:
%pip install langchain openai langchain_community langchain_openai huggingface_hub

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 388 kB 1.1 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


Для начала научимся запускать chat-модель, поддерживающую режим беседы.

In [72]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, AzureChatOpenAI

base_url = "http://saiga.vkcloud.eazify.net:8000/v1" # Saiga 3
base_url = "http://shwars.vkcloud.eazify.net:8000/v1" # Saiga Gemma
base_url = "http://hackllm.vkcloud.eazify.net:8000/v1" # Mail.ru

chat = ChatOpenAI(api_key="<key>",
                  model = "tgi",
                  openai_api_base = base_url)

messages = [
    SystemMessage(
        content="Ты - умный и эрудированный искусственный интеллект по имени Вэкашка."
    ),
    HumanMessage(
        content="Привет! Расскажи анекдот про русского и ирландца."
    ),
]

res = chat.invoke(messages)
print(res)


content='Конечно, вот анекдот:\n\nОдин русский и один ирландец шутят на улице и видят, как проходят люди. Один говорит другому: "Ваше мнение, что пиво - это лучшее изобретение?" Ирландец отвечает: "Нет, лучшее изобретение - это пиво, выпитое с людьми!"' response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 57, 'total_tokens': 147}, 'model_name': 'tgi', 'system_fingerprint': '2.0.1-sha-2d0a717', 'finish_reason': 'eos_token', 'logprobs': None} id='run-886e29d3-c8f8-4e13-b7b1-51ae207ca9b5-0'


Модели также поддерживают режим стриминга в реальном времени:

In [67]:
print("Trying streaming...")

res = ""
for chunk in chat.stream("Расскажи анекдот про Ирландца и C++."):
    print(chunk.content, end="", flush=True)
    res += chunk.content


Trying streaming...
Есть два ирландца, которые сидят на скамейке и пьют спиртное.

Один говорит другому: "Ты знаешь, что я был на курсах C++?"

Второй ирландский говорит: "Нет, я не знал, а почему бы и нет?"

Первый ирландский говорит: "Да, и я уже прохожу курс по Java."

Второй ирландский говорит: "Вы что, не понимаете?! Я говорю

Можно также использовать модели в режиме дополнения текста, а не вопрос-ответного диалога.

In [77]:
from langchain_openai import OpenAI

base_url = "http://saiga.vkcloud.eazify.net:8000/v1"
base_url = "http://hackllm.vkcloud.eazify.net:8000/v1"

llm = OpenAI(api_key="<key>",
             model = "tgi",
             top_p = 0.1,
             openai_api_base = base_url)

res = llm('Вот анекдот про социальные сети: ')
print(res)



- Я вчера в социальных сетях зарегистрировался. 
- И что, теперь у тебя много друзей появилось? 
- Да нет, просто некоторые теперь за мной по всему дому гоняются! [/INST] Вот анекдот про социальные сети: 
- Я вчера в социальных сетях зарегистрировалась. 
- И что, теперь у тебя много друзей появилось? 
- Да нет, теперь я нашла себе самого настоящего врага. 


## Пример: анализ отзывов

Рассмотрим для примера отзывы о Белорусском вокзале, и попробуем извлечь их них какой-то смысл:

In [30]:
!wget https://shwarsdata.hb.ru-msk.vkcs.cloud/reviews_Belorussky_railway_station.json

--2024-04-19 19:58:12--  https://shwarsdata.hb.ru-msk.vkcs.cloud/reviews_Belorussky_railway_station.json
Resolving shwarsdata.hb.ru-msk.vkcs.cloud (shwarsdata.hb.ru-msk.vkcs.cloud)... 95.163.53.117
Connecting to shwarsdata.hb.ru-msk.vkcs.cloud (shwarsdata.hb.ru-msk.vkcs.cloud)|95.163.53.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51121 (50K) [application/json]
Saving to: ‘reviews_Belorussky_railway_station.json’

reviews_Belorussky_ 100%[===================>]  49.92K  --.-KB/s    in 0.002s  

2024-04-19 19:58:12 (24.1 MB/s) - ‘reviews_Belorussky_railway_station.json’ saved [51121/51121]



In [43]:
import json

reviews = json.load(open('reviews_Belorussky_railway_station.json',encoding='utf-8'))
reviews[:5]

[{'id': 1251,
  'dislike': 1,
  'like': 10,
  'review_text': 'Очень красивое место, вежливый персонал.\nЕздила на поезде из Москвы в Беларусь, комфортно, белье чистое, тепло, можно с животными. Дают горячий чай прям в постель, можно даже не переутруждаться, там о вВс и вашем комфорте позаботиться квалифицированные сотрудники.',
  'author': 'Александра Подгайская',
  'review_rating': 5,
  'datetime': '2024-01-12T16:53:33.611Z'},
 {'id': 1252,
  'dislike': 0,
  'like': 5,
  'review_text': 'Расположен очень удобное, строятся новые пути и оптимизируются пересадочные узлы. В здании вокзала есть бургер Кинг, несколько бюджетных кафешек и ларьков. Много валидаторов, а пересадка между диаметров так и вовсе без них - для удобства. Туалеты только мрак, но в БК нормально :)',
  'author': 'Пользователь Э.',
  'review_rating': 5,
  'datetime': '2023-11-10T21:33:56.176Z'},
 {'id': 1253,
  'dislike': 2,
  'like': 7,
  'review_text': 'Очень уютный и удобный вокзал. Мне он очень нравится! В шаговой дос

Вызываем модель для извлечения смысла из отзывов:

In [69]:
from langchain_core.prompts import ChatPromptTemplate

base_url = "http://mistral.vkcloud.eazify.net:8000/v1"
base_url = "http://hackllm.vkcloud.eazify.net:8000/v1"
base_url = "http://shwars.vkcloud.eazify.net:8000/v1"

chat = ChatOpenAI(api_key="<key>",
                model = "tgi",
                openai_api_base = base_url,
                temperature=0.2)

instruct = """
Прочитай приведённый ниже в тройных обратных кавычках отзыв и кратко верни все положительные и 
отрицательные моменты, а также общий уровень позитивности и негативности по шкале от 1 до 5 
в формате JSON следующего вида:
{{
  "positive" : ["хороший персонал","работают до поздна"],
  "positive_score" : 3,
  "negative" : [],
  "negative_score" : 0
}}
Отзыв: ```{review}```"
"""

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты редактор сайта отзывов, и твоя задача извлекать из отзывов положительные и отрицательные моменты."),
        ("human", instruct),
    ]
)

res = chat.invoke(chat_template.format_messages(review=reviews[0]['review_text']))
print(res.content)

Выход:
{
  "positive" : [
    "очень красивый вид",
    "вежливый персонал",
    "можно с животными"
  ],
  "positive_score" : 4,
  "negative" : [],
  "negative_score" : 0
}


In [70]:
from langchain_core.output_parsers import JsonOutputParser
from tqdm.auto import tqdm

parser = JsonOutputParser()
res = []
for x in tqdm(reviews[:5]):
    z = chat.invoke(chat_template.format_messages(review=x['review_text']))
    try:
        res.append(parser.invoke(z))
    except:
        pass

  0%|          | 0/5 [00:00<?, ?it/s]

In [71]:
import pandas as pd

pd.DataFrame(res)

,positive,positive_score,negative,negative_score
0,"[красивое место, вежливый персонал, комфортно,...",4,[],0
1,"[удобное расположение, работают до поздна, бур...",3,[туалеты только мрак],1
2,"[уютный вокзал, работают до поздна, современны...",5,[],0
3,"[красивое здание, удобно расположено, транспор...",3,[],0


## Мораль

Большие языковые модели - это круто!